In [1]:
import pandas as pd
import os 
import copy
from utils.clean_output import clean_epi
from utils.process_new_data import Preprocess_New_Data
from utils.data_processing import get_one_hot, get_splitted_data
from sklearn.model_selection import train_test_split

c:\Users\Rashi\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
old_data = pd.read_csv(os.path.join('data', 'xml_data.csv'))
old_data['epi'] = clean_epi(old_data)
old_data = old_data.drop(columns='soc')

d:\EDUTECH\chatbot\utils\clean_output.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y == i] = '1'
d:\EDUTECH\chatbot\utils\clean_output.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y == i] = '2'
d:\EDUTECH\chatbot\utils\clean_output.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y == i] = '3'
d:\EDUTECH\chatbot\utils\clean_output.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [3]:
#Create new data

new_data = pd.read_excel(os.path.join('data', 'EPI_discourses.xlsx'), sheet_name = None)
new_data['Braga_Gujranwarla'] = new_data['Braga_Gujranwarla'].drop(columns = ['Unnamed: 3', 'Unnamed: 4'])
del new_data['Cuenca_Bielefeld'] 

counter = 0
for key in new_data.keys():
    if key != 'Peshawar_Queretaro':
        if counter == 0:
            epi_data = new_data[key]
            counter = 1
        else:
            epi_data = pd.concat([epi_data, new_data[key]])
            
epi_data = epi_data.drop(columns = 'Person')
epi_data.columns = ['message', 'epi']
epi_data = epi_data.reset_index(drop=True)

self_attr = ['R01', 'R02', 'R03', 'R04', 'R05', '1', '2', '3', '4', '5']
parents_attr = ['R06', 'R07', 'R08', 'R09', 'R10', '6', '7', '8', '9', '10']
teacher_attr = ['R11', 'R12', 'R13', 'R14', 'R15', '11', '12', '13', '14', '15']
re_attr = ['R16', 'R17', '16', '17']
cause_attr = ['R18', 'R19', 'R20', 'R21', '18', '19', '20', '21']

for i in range(len(epi_data)):
    epi = epi_data['epi'][i]
    if epi == epi:
        epi = str(epi)
        epi = epi.replace(",", " ")
        epi = epi.replace("\n", " ")
        epi = epi.split()
        for j in range(len(epi)):
            if epi[j] in self_attr:
                epi[j] = '1'
            elif epi[j] in parents_attr:
                epi[j] = '2'
            elif epi[j] in teacher_attr:
                epi[j] = '3'
            elif epi[j] in re_attr:
                epi[j] = '4'
            elif epi[j] in cause_attr:
                epi[j] = '5'
            else:
                epi[j] = '-1'
        cleaned_epi = list(set(epi))
        if '-1' in cleaned_epi and len(cleaned_epi) > 1:
            cleaned_epi.remove('-1')
        epi_data['epi'][i] = " ".join(cleaned_epi)
    else:
        epi_data['epi'][i] = '-1'


In [4]:
epi_data = pd.concat([old_data, epi_data]).reset_index(drop=True)

In [5]:
def save_epi(epi_data, model_type):
    one_hot = get_one_hot(0, epi_data['epi'])
    epi = get_splitted_data(model_type, one_hot)
    X_train, X_test, y_train, y_test = train_test_split(epi_data['message'], epi, test_size = 0.15, shuffle= True)
    X_train = X_train.reset_index(drop = True)
    train_data = pd.DataFrame(X_train, columns = ['message'])
    train_data['epi'] = y_train
    train_data['epi'] = train_data['epi'].astype(int)
    positive_examples = copy.deepcopy(train_data[train_data['epi'] == 1])
    train_data = train_data.sample(frac=1, weights=len(train_data) - train_data.groupby('epi')['epi'].transform('count'),replace=True)
    train_data = pd.concat([train_data, positive_examples])
    train_data = train_data.sample(frac=1)
    train_data.to_csv(os.path.join('data', model_type+'_train.csv'), index = False)
    
    test_data = pd.DataFrame(X_test, columns = ['message'])
    test_data['epi'] = y_test
    test_data['epi'] = test_data['epi'].astype(int)
    test_data.to_csv(os.path.join('data', model_type+'_test.csv'), index = False)


In [6]:
model_type = ['self', 'parents', 'teacher', 're', 'cause']
for model in model_type:
    save_epi(epi_data, model)